# Source 

- Pytorch 1.6 : https://pytorch.org/docs/stable/
- iterative-stratification : https://github.com/trent-b/iterative-stratification for stratified K fold multilabel
- TabNet : https://arxiv.org/pdf/1908.07442.pdf https://github.com/dreamquark-ai/tabnet

# Approach :
Inference script : 
https://www.kaggle.com/ludovick/introduction-to-tabnet-kfold-10-inference

The Regressor class of TabNet is used as it allows multilabels outputs.
- Stratified K Fold (10 folds) or shufflesplit
- BCE Loss
- TabNet Regressor Model
- Supervised learning




Radu Approach:
* Label Smoothing: 0.001
* Folds: 10
* Patience: 50
* Seed: 3
* **REMOVED** Added Frequency Encoding Variables
* **REMOVED** PCA for Frequency variables (10 components)
* Removed Label Smoothing from vaidation inference
* Added Min Max Clipping
* **REMOVED** Added Extra_features (mean sum std etc.)
* **REMOVED** Drop c62
* **REMOVED** Added Kmeans Clustres
* Stratified validation by drug id (Chris Deotte)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/final-sub-tabnet-v1-training/tabnet_raw_step1_fold2_0.zip
/kaggle/input/final-sub-tabnet-v1-training/tabnet_raw_step1_fold0_0.zip
/kaggle/input/final-sub-tabnet-v1-training/tabnet_raw_step1_fold7_0.zip
/kaggle/input/final-sub-tabnet-v1-training/tabnet_raw_step1_fold6_0.zip
/kaggle/input/final-sub-tabnet-v1-training/custom.css
/kaggle/input/final-sub-tabnet-v1-training/__notebook__.ipynb
/kaggle/input/final-sub-tabnet-v1-training/tabnet_raw_step1_fold1_0.zip
/kaggle/input/final-sub-tabnet-v1-training/tabnet_raw_step1_fold9_0.zip
/kaggle/input/final-sub-tabnet-v1-training/tabnet_raw_step1_fold3_0.zip
/kaggle/input/final-sub-tabnet-v1-training/__results__.html
/kaggle/input/final-sub-tabnet-v1-training/__output__.j

# Install package

In [2]:
! pip install ../input/pytorch16gpu/torch-1.6.0cu101-cp37-cp37m-linux_x86_64.whl

Processing /kaggle/input/pytorch16gpu/torch-1.6.0cu101-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.5.1
    Uninstalling torch-1.5.1:
      Successfully uninstalled torch-1.5.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

kornia 0.3.2 requires torch<1.6.0,>=1.5.0, but you'll have torch 1.6.0+cu101 which is incompatible.
allennlp 1.0.0 requires torch<1.6.0,>=1.5.0, but you'll have torch 1.6.0+cu101 which is incompatible.


In [3]:
!pip install ../input/iterative-stratification/iterative_stratification-0.1.6-py3-none-any.whl


Processing /kaggle/input/iterative-stratification/iterative_stratification-0.1.6-py3-none-any.whl


[](http://)

In [4]:
import sys
sys.path.insert(0, "../input/tabnetdevelop/tabnet-develop")

# What is TabNet ?

TabNet is a Deep Neural Network for tabular data and was designed to learn in a similar way than decision tree based models, in order to have their benefits : ***interpretability*** and ***sparse feature selection***. TabNet uses ***sequential attention to choose which features to reason from at each decision step***, enabling interpretability and better learning (as the learning capacity is used for the most salient/important features). The feature selection is instancewise, so it can be different for each input.

![Example of features selections with multiples steps](https://miro.medium.com/max/700/0*xGrnkDqPkDmC_MWS)

TabNet can use *categoricals* and *numericals features*. There are no global normalization of the inputs, instead of that, a batch normalization is applied. The features obtained is then used at each step. Two parts can be identified at each step (fig a) :
- **<font size="3">the feature selection</font>** : the feature selection is based on a **learnable mask M[i]** which allows soft selection of the important feature. The mask is obtained with the help of **an attentive transformer** (fig d), using the processed features from the previous step a[i-1]. A sparsemax function is used to encourage the sparsity : 
**M[i] = sparsemax(P[i-1] . $h_i$ a[i-1])** where P[i] is the prior scale term and allow to control how much a feature has been used in previous steps; $h_i$ is a trainable function (FC layer) . (More details in the paper)
- **<font size="3">the feature processing</font>** : the features are processed with a feature transformer block (fig c) and then split for the decision step outputs and the information for the subsequence step, such as : 
**[ d[i], a[i]] = $f_i$ (M[i] . f)** with f the inputs data "normalized" by the BN, **M[i]** the learnable mask computed by the attentive transformer, $f_i$ the feature transformer block. **a[i]** is then used in the next step by the attentive transformer, **d[i]** is used to compute the overall decision embedding $d_{out}$, such as **$d_{out}$ = $\sum_{i=0}^{steps}(ReLU(d[i])$** . 

The output is then computed such as : **Outputs = F( $W_{final}$ $d_{out}$ )** with *F activation function (softmax, sigmoid, etc) and $W_{final}$ a FC layer.*
![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F2234817%2Fa08525247089146ce7e1bfdcfe256a2f%2FArchitecture.PNG?generation=1590581898108061&alt=media)
 

# For this competition : 
TabNet can have multiple advantages in this competition:
- it can use gpu, so the training can be quite fast, if we limits the number of epochs and steps
- it can use multilabel prediction, so only one model can be trained contrary to boosting methods.

Even if the model does not give you the best results in CV/LB compared to others approaches, it can still be a good model to add in an ensembling because it is quite fast to train and can add different capabilities, so make the ensembling more exhaustif

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
import numpy as np
import os
import random
import sys
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from tqdm import tqdm
from sklearn.metrics import log_loss
from sklearn.cluster import KMeans

In [6]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(62)

# Data


In [7]:
train = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

remove_vehicle = True

if remove_vehicle:
    train_features = train.loc[train['cp_type']=='trt_cp'].reset_index(drop=True)
    train_targets_scored = train_targets_scored.loc[train['cp_type']=='trt_cp'].reset_index(drop=True)
    train_targets_nonscored = train_targets_nonscored.loc[train['cp_type']=='trt_cp'].reset_index(drop=True)
else:
    train_features = train

In [8]:
train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,trt_cp,72,D1,0.1608,-1.0500,0.2551,-0.2239,-0.2431,0.4256,...,0.0789,0.3538,0.0558,0.3377,-0.4753,-0.2504,-0.7415,0.8413,-0.4259,0.2434
21944,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
21945,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
21946,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [9]:

# ratio for each label

def get_ratio_labels(df):
    columns = list(df.columns)
    columns.pop(0)
    ratios = []
    toremove = []
    for c in columns:
        counts = df[c].value_counts()
        if len(counts) != 1:
            ratios.append(counts[0]/counts[1])
        else:
            toremove.append(c)
    print(f"remove {len(toremove)} columns")
    
    for t in toremove:
        columns.remove(t)
    return columns, np.array(ratios).astype(np.int32)

columns, ratios = get_ratio_labels(train_targets_scored)
columns_nonscored, ratios_nonscored = get_ratio_labels(train_targets_nonscored)

remove 0 columns
remove 71 columns


In [10]:
drop_c62=False
if drop_c62:
    train_features = train_features.drop(['c-62'],axis=1)

# Dataloader

In [11]:
extra_features_=False
add_clusters_= False

def create_cluster(train, test, features, kind = 'g', n_clusters = 35):
    train_ = train[features].copy()
    test_ = test[features].copy()
    data = pd.concat([train_, test_], axis = 0)
    kmeans = KMeans(n_clusters = n_clusters, random_state = 62).fit(data)
    train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
    test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
    #train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
    #test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
    return train, test
    
def transform_data(train, test, col, normalize=True, removed_vehicle=False, extra_features=False, add_clusters=False):
    """
        the first 3 columns represents categories, the others numericals features
    """
    mapping = {"cp_type":{"trt_cp": 0, "ctl_vehicle":1},
               "cp_time":{48:0, 72:1, 24:2},
               "cp_dose":{"D1":0, "D2":1}}
    
    if extra_features:
        features_g = list(train.columns[4:774])
        features_c = list(train.columns[776:876])

        for df in train, test:
            df['g_sum'] = df[features_g].sum(axis = 1)
            df['g_mean'] = df[features_g].mean(axis = 1)
            df['g_std'] = df[features_g].std(axis = 1)
            df['g_kurt'] = df[features_g].kurtosis(axis = 1)
            df['g_skew'] = df[features_g].skew(axis = 1)
            df['c_sum'] = df[features_c].sum(axis = 1)
            df['c_mean'] = df[features_c].mean(axis = 1)
            df['c_std'] = df[features_c].std(axis = 1)
            df['c_kurt'] = df[features_c].kurtosis(axis = 1)
            df['c_skew'] = df[features_c].skew(axis = 1)
            df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
            df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
            df['gc_std'] = df[features_g + features_c].std(axis = 1)
            df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
            df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
    if add_clusters:
        features_g = list(train.columns[4:774])
        features_c = list(train.columns[776:876])
    
        train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = 35)
        train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = 5)
        
    
    if removed_vehicle:
        categories_tr = np.stack([ train[c].apply(lambda x: mapping[c][x]).values for c in col[1:3]], axis=1)
        categories_test = np.stack([ test[c].apply(lambda x: mapping[c][x]).values for c in col[1:3]], axis=1)
    else:
        categories_tr = np.stack([ train[c].apply(lambda x: mapping[c][x]).values for c in col[:3]], axis=1)
        categories_test = np.stack([ test[c].apply(lambda x: mapping[c][x]).values for c in col[:3]], axis=1)
    if add_clusters:
        categories_tr = np.append(categories_tr, train[['clusters_g', 'clusters_c']].values, axis=1)
        categories_test = np.append(categories_test, test[['clusters_g', 'clusters_c']].values, axis=1)
        
    #categories_tr = np.append(categories_tr, train[['sig_id']].values, axis=1) 
    #categories_test = np.append(categories_test, test[['sig_id']].values, axis=1)
    
    max_ = 10.
    min_ = -10.
   
    if removed_vehicle:
        numerical_tr = train[col[3:]].values
        numerical_test = test[col[3:]].values
    else:
        numerical_tr = train[col[3:]].values
        numerical_test = test[col[3:]].values
    if normalize:
        numerical_tr = (numerical_tr-min_)/(max_ - min_)
        numerical_test = (numerical_test-min_)/(max_ - min_)
    return categories_tr, categories_test, numerical_tr, numerical_test
if extra_features_:
    col_features = list(train_features.columns)[1:] + ['g_sum', 'g_mean', 'g_std', 'g_kurt', 'g_skew', 'c_sum', 'c_mean', 'c_std', 'c_kurt', 'c_skew', 'gc_sum', 'gc_mean', 'gc_std', 'gc_kurt', 'gc_skew']  
else:
    col_features = list(train_features.columns)[1:]
cat_tr, cat_test, numerical_tr, numerical_test = transform_data(train_features, test_features, col_features, normalize=False, removed_vehicle=remove_vehicle, extra_features=extra_features_, add_clusters=add_clusters_)
targets_tr = train_targets_scored[columns].values.astype(np.float32)
targets2_tr = train_targets_nonscored[columns_nonscored].values.astype(np.float32)


In [12]:
freq_encodeing = False
if freq_encodeing:
    df_numerical_tr = pd.DataFrame(numerical_tr)
    df_numerical_test = pd.DataFrame(numerical_test)
    df_numerical_count_tr = pd.DataFrame()
    df_numerical_count_test = pd.DataFrame()
    df_numerical_tr_map = pd.DataFrame()
    df_numerical_test_map = pd.DataFrame()

    def myround(x, prec=2, base=.05):
      return round(base * round(float(x)/base),prec)

    for col in df_numerical_tr.columns:
        df_numerical_count_tr.loc[:,col] = df_numerical_tr.loc[:,col].apply(lambda x: myround(x)).value_counts()
        df_numerical_tr_map.loc[:,col] = df_numerical_tr.loc[:,col].apply(lambda x: myround(x)).map(df_numerical_count_tr.loc[:,col])
        df_numerical_count_test.loc[:,col] = df_numerical_test.loc[:,col].apply(lambda x: myround(x)).value_counts()
        df_numerical_test_map.loc[:,col] = df_numerical_test.loc[:,col].apply(lambda x: myround(x)).map(df_numerical_count_test.loc[:,col])
    df_numerical_tr_map = df_numerical_tr_map.fillna(0)
    df_numerical_test_map = df_numerical_test_map.fillna(0)

    train_data = np.concatenate([cat_tr, numerical_tr, df_numerical_tr_map.values], axis=1)
    print(train_data.shape)
    test_data = np.concatenate([cat_test, numerical_test, df_numerical_test_map.values], axis=1)
    print(test_data.shape)

    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    train_data[:,cat_tr.shape[1]:] = scaler.fit_transform(train_data[:,cat_tr.shape[1]:])
    test_data[:,cat_tr.shape[1]:]= scaler.transform(test_data[:,cat_tr.shape[1]:])
else:
    train_data = np.concatenate([cat_tr, numerical_tr], axis=1)
    print(train_data.shape)
    test_data = np.concatenate([cat_test, numerical_test], axis=1)
    print(test_data.shape)

(21948, 874)
(3982, 874)


In [13]:
Trans_PCA=False
if Trans_PCA:
    from sklearn.decomposition import PCA
    n_comp = 10
    PCAF = PCA(n_components=n_comp, random_state=62)
    numerical_tr_PCA = PCAF.fit_transform(train_data[:,-df_numerical_tr_map.shape[1]:])
    numerical_test_PCA = PCAF.transform(test_data[:,-df_numerical_tr_map.shape[1]:])
    
    train_data = train_data[:,:-df_numerical_tr_map.shape[1]]
    test_data = test_data[:,:-df_numerical_tr_map.shape[1]]
    
    train_data = np.concatenate([train_data, numerical_tr_PCA], axis=1)
    print(train_data.shape)
    test_data = np.concatenate([test_data, numerical_test_PCA], axis=1)
    print(test_data.shape)
    print(PCAF.explained_variance_ratio_)
    print(PCAF.explained_variance_ratio_.sum())

    

# Utils

In [14]:
           
def inference_fn(model, X ,verbose=True):
    with torch.no_grad():
        y_preds = model.predict( X )
        y_preds = torch.sigmoid(torch.as_tensor(y_preds)).numpy()
    return y_preds

In [15]:
def log_loss_score(actual, predicted,  eps=1e-15):

        """
        :param predicted:   The predicted probabilities as floats between 0-1
        :param actual:      The binary labels. Either 0 or 1.
        :param eps:         Log(0) is equal to infinity, so we need to offset our predicted values slightly by eps from 0 or 1
        :return:            The logarithmic loss between between the predicted probability assigned to the possible outcomes for item i, and the actual outcome.
        """

        
        p1 = actual * np.log(predicted+eps)
        p0 = (1-actual) * np.log(1-predicted+eps)
        loss = p0 + p1

        return -loss.mean()

In [16]:
def log_loss_multi(y_true, y_pred):
    M = y_true.shape[1]
    results = np.zeros(M)
    for i in range(M):
        results[i] = log_loss_score(y_true[:,i], y_pred[:,i])
    return results.mean()
        

In [17]:
def check_targets(targets):
    ### check if targets are all binary in training set
    
    for i in range(targets.shape[1]):
        if len(np.unique(targets[:,i])) != 2:
            return False
    return True

In [18]:
def auc_multi(y_true, y_pred):
    M = y_true.shape[1]
    results = np.zeros(M)
    for i in range(M):
        try:
            results[i] = roc_auc_score(y_true[:,i], y_pred[:,i])
        except:
            pass
    return results.mean()

# Model

Cross Entrpoy with Smoothing

In [19]:
## TABNET
#from pytorch_tabnet.tab_model import TabModel
import torch
import numpy as np
from scipy.sparse import csc_matrix
import time
from abc import abstractmethod
from pytorch_tabnet import tab_network
from pytorch_tabnet.multiclass_utils import unique_labels
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score
from torch.nn.utils import clip_grad_norm_
from pytorch_tabnet.utils import (PredictDataset,
                                  create_dataloaders,
                                  create_explain_matrix)
from sklearn.base import BaseEstimator
from torch.utils.data import DataLoader
from copy import deepcopy
import io
import json
from pathlib import Path
import shutil
import zipfile


class TabModel(BaseEstimator):
    def __init__(self, n_d=8, n_a=8, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1,
                 n_independent=2, n_shared=2, epsilon=1e-15,  momentum=0.02,
                 lambda_sparse=1e-3, seed=0,
                 clip_value=1, verbose=1,
                 optimizer_fn=torch.optim.Adam,
                 optimizer_params=dict(lr=2e-2),
                 scheduler_params=None, scheduler_fn=None,
                 mask_type="sparsemax",
                 input_dim=None, output_dim=None,
                 device_name='auto',
                 label_smoothing = 0.001):
        """ Class for TabNet model
        Parameters
        ----------
            device_name: str
                'cuda' if running on GPU, 'cpu' if not, 'auto' to autodetect
        """

        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.cat_idxs = cat_idxs
        self.cat_dims = cat_dims
        self.cat_emb_dim = cat_emb_dim
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.epsilon = epsilon
        self.momentum = momentum
        self.lambda_sparse = lambda_sparse
        self.clip_value = clip_value
        self.verbose = verbose
        self.optimizer_fn = optimizer_fn
        self.optimizer_params = optimizer_params
        self.device_name = device_name
        self.scheduler_params = scheduler_params
        self.scheduler_fn = scheduler_fn
        self.mask_type = mask_type
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.batch_size = 1024
        
        #Adaugat de Radu:
        self.label_smoothing = label_smoothing

        self.seed = seed
        torch.manual_seed(self.seed)
        # Defining device
        if device_name == 'auto':
            if torch.cuda.is_available():
                device_name = 'cuda'
            else:
                device_name = 'cpu'
        self.device = torch.device(device_name)
        print(f"Device used : {self.device}")

    @abstractmethod
    def construct_loaders(self, X_train, y_train, X_valid, y_valid,
                          weights, batch_size, num_workers, drop_last):
        """
        Returns
        -------
        train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
            Training and validation dataloaders
        -------
        """
        raise NotImplementedError('users must define construct_loaders to use this base class')

    def init_network(
                     self,
                     input_dim,
                     output_dim,
                     n_d,
                     n_a,
                     n_steps,
                     gamma,
                     cat_idxs,
                     cat_dims,
                     cat_emb_dim,
                     n_independent,
                     n_shared,
                     epsilon,
                     virtual_batch_size,
                     momentum,
                     device_name,
                     mask_type,
                     ):
        self.network = tab_network.TabNet(
            input_dim,
            output_dim,
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            cat_idxs=cat_idxs,
            cat_dims=cat_dims,
            cat_emb_dim=cat_emb_dim,
            n_independent=n_independent,
            n_shared=n_shared,
            epsilon=epsilon,
            virtual_batch_size=virtual_batch_size,
            momentum=momentum,
            device_name=device_name,
            mask_type=mask_type).to(self.device)

        self.reducing_matrix = create_explain_matrix(
            self.network.input_dim,
            self.network.cat_emb_dim,
            self.network.cat_idxs,
            self.network.post_embed_dim)

    def fit(self, X_train, y_train, X_valid=None, y_valid=None, loss_fn=None,
            weights=0, max_epochs=100, patience=10, batch_size=1024,
            virtual_batch_size=128, num_workers=0, drop_last=False, label_smoothing=0.001):
        """Train a neural network stored in self.network
        Using train_dataloader for training data and
        valid_dataloader for validation.
        Parameters
        ----------
            X_train: np.ndarray
                Train set
            y_train : np.array
                Train targets
            X_train: np.ndarray
                Train set
            y_train : np.array
                Train targets
            weights : bool or dictionnary
                0 for no balancing
                1 for automated balancing
                dict for custom weights per class
            max_epochs : int
                Maximum number of epochs during training
            patience : int
                Number of consecutive non improving epoch before early stopping
            batch_size : int
                Training batch size
            virtual_batch_size : int
                Batch size for Ghost Batch Normalization (virtual_batch_size < batch_size)
            num_workers : int
                Number of workers used in torch.utils.data.DataLoader
            drop_last : bool
                Whether to drop last batch during training
        """
        # update model name

        self.update_fit_params(X_train, y_train, X_valid, y_valid, loss_fn,
                               weights, max_epochs, patience, batch_size,
                               virtual_batch_size, num_workers, drop_last, label_smoothing)

        train_dataloader, valid_dataloader = self.construct_loaders(X_train,
                                                                    y_train,
                                                                    X_valid,
                                                                    y_valid,
                                                                    self.updated_weights,
                                                                    self.batch_size,
                                                                    self.num_workers,
                                                                    self.drop_last)

        self.init_network(
            input_dim=self.input_dim,
            output_dim=self.output_dim,
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            cat_idxs=self.cat_idxs,
            cat_dims=self.cat_dims,
            cat_emb_dim=self.cat_emb_dim,
            n_independent=self.n_independent,
            n_shared=self.n_shared,
            epsilon=self.epsilon,
            virtual_batch_size=self.virtual_batch_size,
            momentum=self.momentum,
            device_name=self.device_name,
            mask_type=self.mask_type
        )

        self.optimizer = self.optimizer_fn(self.network.parameters(),
                                           **self.optimizer_params)

        if self.scheduler_fn:
            self.scheduler = self.scheduler_fn(self.optimizer, **self.scheduler_params)
        else:
            self.scheduler = None

        self.losses_train = []
        self.losses_valid = []
        self.learning_rates = []
        self.metrics_train = []
        self.metrics_valid = []

        if self.verbose > 0:
            print("Will train until validation stopping metric",
                  f"hasn't improved in {self.patience} rounds.")
            msg_epoch = f'| EPOCH |  train  |   valid  | total time (s)'
            print('---------------------------------------')
            print(msg_epoch)

        total_time = 0
        while (self.epoch < self.max_epochs and
               self.patience_counter < self.patience):
            starting_time = time.time()
            # updates learning rate history
            self.learning_rates.append(self.optimizer.param_groups[-1]["lr"])

            fit_metrics = self.fit_epoch(train_dataloader, valid_dataloader)

            # leaving it here, may be used for callbacks later
            self.losses_train.append(fit_metrics['train']['loss_avg'])
            self.losses_valid.append(fit_metrics['valid']['total_loss'])
            self.metrics_train.append(fit_metrics['train']['stopping_loss'])
            self.metrics_valid.append(fit_metrics['valid']['stopping_loss'])

            stopping_loss = fit_metrics['valid']['stopping_loss']
            if stopping_loss < self.best_cost:
                self.best_cost = stopping_loss
                self.patience_counter = 0
                # Saving model
                self.best_network = deepcopy(self.network)
                has_improved = True
            else:
                self.patience_counter += 1
                has_improved=False
            self.epoch += 1
            total_time += time.time() - starting_time
            if self.verbose > 0:
                if self.epoch % self.verbose == 0:
                    separator = "|"
                    msg_epoch = f"| {self.epoch:<5} | "
                    msg_epoch += f" {fit_metrics['train']['stopping_loss']:.5f}"
                    msg_epoch += f' {separator:<2} '
                    msg_epoch += f" {fit_metrics['valid']['stopping_loss']:.5f}"
                    msg_epoch += f' {separator:<2} '
                    msg_epoch += f" {np.round(total_time, 1):<10}"
                    msg_epoch += f" {has_improved}"
                    print(msg_epoch)

        if self.verbose > 0:
            if self.patience_counter == self.patience:
                print(f"Early stopping occured at epoch {self.epoch}")
            print(f"Training done in {total_time:.3f} seconds.")
            print('---------------------------------------')

        self.history = {"train": {"loss": self.losses_train,
                                  "metric": self.metrics_train,
                                  "lr": self.learning_rates},
                        "valid": {"loss": self.losses_valid,
                                  "metric": self.metrics_valid}}
        # load best models post training
        self.load_best_model()

        # compute feature importance once the best model is defined
        self._compute_feature_importances(train_dataloader)

    def save_model(self, path):
        """
        Saving model with two distinct files.
        """
        saved_params = {}
        for key, val in self.get_params().items():
            if isinstance(val, type):
                # Don't save torch specific params
                continue
            else:
                saved_params[key] = val

        # Create folder
        Path(path).mkdir(parents=True, exist_ok=True)

        # Save models params
        with open(Path(path).joinpath("model_params.json"), "w", encoding="utf8") as f:
            json.dump(saved_params, f)

        # Save state_dict
        torch.save(self.network.state_dict(), Path(path).joinpath("network.pt"))
        shutil.make_archive(path, 'zip', path)
        shutil.rmtree(path)
        print(f"Successfully saved model at {path}.zip")
        return f"{path}.zip"

    def load_model(self, filepath):

        try:
            with zipfile.ZipFile(filepath) as z:
                with z.open("model_params.json") as f:
                    loaded_params = json.load(f)
                with z.open("network.pt") as f:
                    try:
                        saved_state_dict = torch.load(f)
                    except io.UnsupportedOperation:
                        # In Python <3.7, the returned file object is not seekable (which at least
                        # some versions of PyTorch require) - so we'll try buffering it in to a
                        # BytesIO instead:
                        saved_state_dict = torch.load(io.BytesIO(f.read()))
        except KeyError:
            raise KeyError("Your zip file is missing at least one component")

        self.__init__(**loaded_params)

        self.init_network(
            input_dim=self.input_dim,
            output_dim=self.output_dim,
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            cat_idxs=self.cat_idxs,
            cat_dims=self.cat_dims,
            cat_emb_dim=self.cat_emb_dim,
            n_independent=self.n_independent,
            n_shared=self.n_shared,
            epsilon=self.epsilon,
            virtual_batch_size=1024,
            momentum=self.momentum,
            device_name=self.device_name,
            mask_type=self.mask_type
        )
        self.network.load_state_dict(saved_state_dict)
        self.network.eval()
        return

    def fit_epoch(self, train_dataloader, valid_dataloader):
        """
        Evaluates and updates network for one epoch.
        Parameters
        ----------
            train_dataloader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
            valid_dataloader: a :class: `torch.utils.data.Dataloader`
                DataLoader with valid set
        """
        train_metrics = self.train_epoch(train_dataloader)
        valid_metrics = self.predict_epoch(valid_dataloader)

        fit_metrics = {'train': train_metrics,
                       'valid': valid_metrics}

        return fit_metrics

    @abstractmethod
    def train_epoch(self, train_loader):
        """
        Trains one epoch of the network in self.network
        Parameters
        ----------
            train_loader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
        """
        raise NotImplementedError('users must define train_epoch to use this base class')

    @abstractmethod
    def train_batch(self, data, targets):
        """
        Trains one batch of data
        Parameters
        ----------
            data: a :tensor: `torch.tensor`
                Input data
            target: a :tensor: `torch.tensor`
                Target data
        """
        raise NotImplementedError('users must define train_batch to use this base class')

    @abstractmethod
    def predict_epoch(self, loader):
        """
        Validates one epoch of the network in self.network
        Parameters
        ----------
            loader: a :class: `torch.utils.data.Dataloader`
                    DataLoader with validation set
        """
        raise NotImplementedError('users must define predict_epoch to use this base class')

    @abstractmethod
    def predict_batch(self, data, targets):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            batch_outs: dict
        """
        raise NotImplementedError('users must define predict_batch to use this base class')

    def load_best_model(self):
        if self.best_network is not None:
            self.network = self.best_network

    @abstractmethod
    def predict(self, X):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            predictions: np.array
                Predictions of the regression problem or the last class
        """
        raise NotImplementedError('users must define predict to use this base class')

    def explain(self, X):
        """
        Return local explanation
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            M_explain: matrix
                Importance per sample, per columns.
            masks: matrix
                Sparse matrix showing attention masks used by network.
        """
        self.network.eval()

        dataloader = DataLoader(PredictDataset(X),
                                batch_size=self.batch_size, shuffle=False)

        for batch_nb, data in enumerate(dataloader):
            data = data.to(self.device).float()

            M_explain, masks = self.network.forward_masks(data)
            for key, value in masks.items():
                masks[key] = csc_matrix.dot(value.cpu().detach().numpy(),
                                            self.reducing_matrix)

            if batch_nb == 0:
                res_explain = csc_matrix.dot(M_explain.cpu().detach().numpy(),
                                             self.reducing_matrix)
                res_masks = masks
            else:
                res_explain = np.vstack([res_explain,
                                         csc_matrix.dot(M_explain.cpu().detach().numpy(),
                                                        self.reducing_matrix)])
                for key, value in masks.items():
                    res_masks[key] = np.vstack([res_masks[key], value])
        return res_explain, res_masks

    def _compute_feature_importances(self, loader):
        self.network.eval()
        feature_importances_ = np.zeros((self.network.post_embed_dim))
        for data, targets in loader:
            data = data.to(self.device).float()
            M_explain, masks = self.network.forward_masks(data)
            feature_importances_ += M_explain.sum(dim=0).cpu().detach().numpy()

        feature_importances_ = csc_matrix.dot(feature_importances_,
                                              self.reducing_matrix)
        self.feature_importances_ = feature_importances_ / np.sum(feature_importances_)
        
        
class TabNetRegressor(TabModel):

    def construct_loaders(self, X_train, y_train, X_valid, y_valid, weights,
                          batch_size, num_workers, drop_last):
        """
        Returns
        -------
        train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
            Training and validation dataloaders
        -------
        """
        if isinstance(weights, int):
            if weights == 1:
                raise ValueError("Please provide a list of weights for regression.")
        if isinstance(weights, dict):
            raise ValueError("Please provide a list of weights for regression.")

        train_dataloader, valid_dataloader = create_dataloaders(X_train,
                                                                y_train,
                                                                X_valid,
                                                                y_valid,
                                                                weights,
                                                                batch_size,
                                                                num_workers,
                                                                drop_last)
        return train_dataloader, valid_dataloader

    def update_fit_params(self, X_train, y_train, X_valid, y_valid, loss_fn,
                          weights, max_epochs, patience,
                          batch_size, virtual_batch_size, num_workers, drop_last,label_smoothing):

        if loss_fn is None:
            self.loss_fn = torch.nn.functional.mse_loss
        else:
            self.loss_fn = loss_fn

        assert X_train.shape[1] == X_valid.shape[1], "Dimension mismatch X_train X_valid"
        self.input_dim = X_train.shape[1]

        if len(y_train.shape) == 1:
            raise ValueError("""Please apply reshape(-1, 1) to your targets
                                if doing single regression.""")
        assert y_train.shape[1] == y_valid.shape[1], "Dimension mismatch y_train y_valid"
        self.output_dim = y_train.shape[1]

        self.updated_weights = weights

        self.max_epochs = max_epochs
        self.patience = patience
        self.batch_size = batch_size
        self.virtual_batch_size = virtual_batch_size
        # Initialize counters and histories.
        self.patience_counter = 0
        self.epoch = 0
        self.best_cost = np.inf
        self.num_workers = num_workers
        self.drop_last = drop_last
        self.label_smoothing = label_smoothing

    def train_epoch(self, train_loader):
        """
        Trains one epoch of the network in self.network
        Parameters
        ----------
            train_loader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
        """

        self.network.train()
        y_preds = []
        ys = []
        total_loss = 0

        for data, targets in train_loader:
            batch_outs = self.train_batch(data, targets)
            y_preds.append(batch_outs["y_preds"].cpu().detach().numpy())
            ys.append(batch_outs["y"].cpu().detach().numpy())
            total_loss += batch_outs["loss"]

        y_preds = np.vstack(y_preds)
        ys = np.vstack(ys)

        #stopping_loss = mean_squared_error(y_true=ys, y_pred=y_preds)
        stopping_loss =log_loss_multi(ys, torch.sigmoid(torch.as_tensor(y_preds)).numpy()  )
        total_loss = total_loss / len(train_loader)
        epoch_metrics = {'loss_avg': total_loss,
                         'stopping_loss': total_loss,
                         }

        if self.scheduler is not None:
            self.scheduler.step()
        return epoch_metrics

    def train_batch(self, data, targets):
        """
        Trains one batch of data
        Parameters
        ----------
            data: a :tensor: `torch.tensor`
                Input data
            target: a :tensor: `torch.tensor`
                Target data
        """
        self.network.train()
        data = data.to(self.device).float()

        targets = targets.to(self.device).float()
        self.optimizer.zero_grad()

        output, M_loss = self.network(data)
        
        #Added by Radu:
        y_smo = targets * (1 - self.label_smoothing) + 0.5 * self.label_smoothing

        loss = self.loss_fn(output, y_smo)
        
        loss -= self.lambda_sparse*M_loss

        loss.backward()
        if self.clip_value:
            clip_grad_norm_(self.network.parameters(), self.clip_value)
        self.optimizer.step()

        loss_value = loss.item()
        batch_outs = {'loss': loss_value,
                      'y_preds': output,
                      'y': targets}
        return batch_outs

    def predict_epoch(self, loader):
        """
        Validates one epoch of the network in self.network
        Parameters
        ----------
            loader: a :class: `torch.utils.data.Dataloader`
                    DataLoader with validation set
        """
        y_preds = []
        ys = []
        self.network.eval()
        total_loss = 0

        for data, targets in loader:
            batch_outs = self.predict_batch(data, targets)
            total_loss += batch_outs["loss"]
            y_preds.append(batch_outs["y_preds"].cpu().detach().numpy())
            ys.append(batch_outs["y"].cpu().detach().numpy())

        y_preds = np.vstack(y_preds)
        ys = np.vstack(ys)

        stopping_loss =log_loss_multi(ys, torch.sigmoid(torch.as_tensor(y_preds)).numpy()  ) #mean_squared_error(y_true=ys, y_pred=y_preds)

        total_loss = total_loss / len(loader)
        epoch_metrics = {'total_loss': total_loss,
                         'stopping_loss': stopping_loss}

        return epoch_metrics

    def predict_batch(self, data, targets):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            batch_outs: dict
        """
        self.network.eval()
        data = data.to(self.device).float()
        targets = targets.to(self.device).float()

        output, M_loss = self.network(data)
       
        #y_smo = targets * (1 - self.label_smoothing) + 0.5 * self.label_smoothing
        loss = self.loss_fn(output, targets)
        #print(self.loss_fn, loss)
        loss -= self.lambda_sparse*M_loss
        #print(loss)
        loss_value = loss.item()
        batch_outs = {'loss': loss_value,
                      'y_preds': output,
                      'y': targets}
        return batch_outs

    def predict(self, X):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            predictions: np.array
                Predictions of the regression problem
        """
        self.network.eval()
        dataloader = DataLoader(PredictDataset(X),
                                batch_size=self.batch_size, shuffle=False)

        results = []
        for batch_nb, data in enumerate(dataloader):
            data = data.to(self.device).float()

            output, M_loss = self.network(data)
            predictions = output.cpu().detach().numpy()
            results.append(predictions)
        res = np.vstack(results)
        return res

# script

In [20]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit
from sklearn.metrics import roc_auc_score

In [21]:
class Config(object):
    def __init__(self):
        self.num_class = targets_tr.shape[1]
        self.verbose=False
        self.seed = 64
        self.SPLITS= 10
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.EPOCHS = 200
        self.num_ensembling = 1
        # Parameters model
        self.cat_emb_dim=[1] * cat_tr.shape[1] #to choose
        self.cats_idx = list(range(cat_tr.shape[1]))
        self.cat_dims = [len(np.unique(cat_tr[:, i])) for i in range(cat_tr.shape[1])]
        #self.num_numericals= numerical_tr.shape[1]
    
        # save
        self.save_name = "../input/final-sub-tabnet-v1-training/tabnet_raw_step1"
        
        self.strategy = "KFOLD" # 
cfg = Config()


In [22]:
#X_test = np.concatenate([cat_test, numerical_test ], axis=1)

In [23]:
SEED = 64
NFOLDS = cfg.SPLITS

# LOAD LIBRARIES (from PIP or Kaggle Dataset) 
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# LOAD FILES
scored = train_targets_scored.copy()
drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
targets = scored.columns[1:]
scored = scored.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=SEED)
tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=SEED)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
scored['kfold'] = scored.drug_id.map(dct1)
scored.loc[scored.kfold.isna(),'kfold'] = scored.loc[scored.kfold.isna(),'sig_id'].map(dct2)
scored.kfold = scored.kfold.astype('int8')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [24]:
scored.shape

(21948, 209)

In [25]:
train_data = pd.merge(pd.DataFrame(train_data), scored[['kfold']], how='left', left_index=True, right_index=True)
targets_tr = pd.merge(pd.DataFrame(targets_tr), scored[['kfold']], how='left', left_index=True, right_index=True)

In [26]:
for seed in range(2):
    print(seed)

0
1


In [27]:
if cfg.strategy == "KFOLD":
    oof_preds_all = []
    oof_targets_all = []
    scores_all =  []
    scores_auc_all= []
    preds_test = []
    for seed in range(cfg.num_ensembling):
        print("## SEED : ", seed)
        #mskf = MultilabelStratifiedKFold(n_splits=cfg.SPLITS, random_state=cfg.seed+seed, shuffle=True)
        oof_preds = []
        oof_targets = []
        scores = []
        scores_auc = []
        p = []
        #for j, (train_idx, val_idx) in enumerate(mskf.split(np.zeros(len(cat_tr)), targets_tr)):
        for j in range(cfg.SPLITS):
            print("FOLDS : ", j)

            ## model
            X_train, y_train = torch.as_tensor(train_data[train_data['kfold'] != j].drop(columns = ['kfold']).values), torch.as_tensor(targets_tr[targets_tr['kfold'] != j].drop(columns = ['kfold']).values)
            X_val, y_val = torch.as_tensor(train_data[train_data['kfold'] == j].drop(columns = ['kfold']).values), torch.as_tensor(targets_tr[targets_tr['kfold'] == j].drop(columns = ['kfold']).values)
            model = TabNetRegressor(n_d=24, n_a=24, n_steps=1, gamma=1.3, lambda_sparse=0, cat_dims=cfg.cat_dims, cat_emb_dim=cfg.cat_emb_dim, cat_idxs=cfg.cats_idx, optimizer_fn=torch.optim.Adam,
                                   optimizer_params=dict(lr=2e-2, weight_decay=1e-5), mask_type='entmax', device_name=cfg.device, scheduler_params=dict(milestones=[100,150], gamma=0.9), scheduler_fn=torch.optim.lr_scheduler.MultiStepLR)
            #'sparsemax'
            
            #model.fit(X_train=X_train, y_train=y_train, X_valid=X_val, y_valid=y_val,max_epochs=cfg.EPOCHS, patience=50, batch_size=1024, virtual_batch_size=128,
                    #num_workers=0, drop_last=False, loss_fn=torch.nn.functional.binary_cross_entropy_with_logits, label_smoothing = 0.001)
            #model.load_best_model()
            
            name = cfg.save_name + f"_fold{j}_{seed}.zip"
            model.load_model(name)
            
            preds = model.predict(X_val)
            preds = torch.sigmoid(torch.as_tensor(preds)).detach().cpu().numpy()
            score = log_loss_multi(y_val, preds)
            #name = cfg.save_name + f"_fold{j}_{seed}"
            #model.save_model(name)
            ## preds on test
            temp = model.predict(test_data)
            p.append(torch.sigmoid(torch.as_tensor(temp)).detach().cpu().numpy())
            ## save oof to compute the CV later
            oof_preds.append(preds)
            oof_targets.append(y_val)
            scores.append(score)
            scores_auc.append(auc_multi(y_val,preds))
            print(f"validation fold {j} : {score}")
            
        p = np.stack(p)
        preds_test.append(p)    
        oof_preds_all.append(np.concatenate(oof_preds))
        oof_targets_all.append(np.concatenate(oof_targets))
        scores_all.append(np.array(scores))
        scores_auc_all.append(np.array(scores_auc))
    preds_test = np.stack(preds_test)

## SEED :  0
FOLDS :  0
Device used : cuda
Device used : cuda
validation fold 0 : 0.017789696181052844
FOLDS :  1
Device used : cuda
Device used : cuda
validation fold 1 : 0.017838912915994494
FOLDS :  2
Device used : cuda
Device used : cuda
validation fold 2 : 0.017682345672476656
FOLDS :  3
Device used : cuda
Device used : cuda
validation fold 3 : 0.017358052611545414
FOLDS :  4
Device used : cuda
Device used : cuda
validation fold 4 : 0.017978976226074948
FOLDS :  5
Device used : cuda
Device used : cuda
validation fold 5 : 0.017936706252213008
FOLDS :  6
Device used : cuda
Device used : cuda
validation fold 6 : 0.016955487434051533
FOLDS :  7
Device used : cuda
Device used : cuda
validation fold 7 : 0.01751639734296969
FOLDS :  8
Device used : cuda
Device used : cuda
validation fold 8 : 0.01692219076817177
FOLDS :  9
Device used : cuda
Device used : cuda
validation fold 9 : 0.017890024295888245


In [28]:
if cfg.strategy == "KFOLD":

    for i in range(cfg.num_ensembling): 
        print("CV score fold : ", log_loss_multi(oof_targets_all[i], oof_preds_all[i]))
        print("auc mean : ", sum(scores_auc_all[i])/len(scores_auc_all[i]))


CV score fold :  0.017585692541827493
auc mean :  0.457787884513296


In [29]:
print(scores_all)

[array([0.0177897 , 0.01783891, 0.01768235, 0.01735805, 0.01797898,
       0.01793671, 0.01695549, 0.0175164 , 0.01692219, 0.01789002])]


In [30]:
print(model)

TabNetRegressor(cat_dims=[3, 2], cat_emb_dim=[1, 1], cat_idxs=[0, 1],
                device_name='cuda', input_dim=874, lambda_sparse=0,
                mask_type='entmax', n_a=24, n_d=24, n_steps=1,
                optimizer_params={'lr': 0.02, 'weight_decay': 1e-05},
                output_dim=206,
                scheduler_params={'gamma': 0.9, 'milestones': [100, 150]})


In [31]:
p_min = 0.0005
p_max = 0.9995

submission[columns] = np.clip(preds_test.mean(1).mean(0), p_min, p_max)
submission.loc[test_features['cp_type']=='ctl_vehicle', submission.columns[1:]] = 0
submission.to_csv("submission.csv", index=False)